In [ ]:
# 1. 라이브러리 불러오기
import pandas as pd
import numpy as np
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.metrics.pairwise import cosine_similarity
import math
import os
import sys
from enum import Enum
import datetime
import pickle
import fire

# 2. CSV 파일 경로 지정 
file_path = "popular_games.csv"

# 3. CSV 불러오기
df = pd.read_csv(file_path)

# 4. 불필요한 인덱스 컬럼 제거
if "Unnamed: 0" in df.columns:
    df = df.drop(columns=["Unnamed: 0"])

# 5. 상위 5개 데이터 표시
df.head()


# 1. 장르 라벨 인코딩
le = LabelEncoder()
df["genre_encoded"] = le.fit_transform(df["genre"])

# 2. 수치형 피처 정규화
scaler = StandardScaler()
scaled_features = scaler.fit_transform(df[["playtime", "rating", "owned_ratio"]])

# 3. 장르 임베딩 
# 장르 임베딩 차원 = 4 (csv 내 장르 수 총 9개인 바, 4개로 설정)
embedding_dim = 4
np.random.seed(42)
genre_embeddings = np.random.randn(len(le.classes_), embedding_dim)

# 각 게임의 장르 벡터 가져오기
genre_vectors = np.array([genre_embeddings[idx] for idx in df["genre_encoded"]])

# 4. 최종 게임 벡터 = [장르 임베딩 | 수치형 피처]
game_vectors = np.hstack([genre_vectors, scaled_features])

# 5. 추천 함수
def recommend(game_name, top_n=5):
    if game_name not in df["game_name"].values:
        return f"게임 '{game_name}'을(를) 찾을 수 없습니다."
    idx = df[df["game_name"] == game_name].index[0]
    target_vec = game_vectors[idx].reshape(1, -1)
    
    sims = cosine_similarity(target_vec, game_vectors)[0]
    similar_idx = sims.argsort()[::-1][1:top_n+1]  # 자기 자신 제외
    return df.iloc[similar_idx][["game_name", "genre", "rating", "owned_ratio"]]

# 예시 실행
print(recommend("Devil May Cry 3: Dante's Awakening Special Edition", top_n=5))


class SimpleDataLoader:
    def __init__(self, features, labels, batch_size=32, shuffle=True):
        self.features = np.array(features)
        self.labels = np.array(labels)
        self.batch_size = batch_size
        self.shuffle = shuffle
        self.num_samples = len(features)
        self.indices = np.arange(self.num_samples)

    def __iter__(self):
        if self.shuffle:
            np.random.shuffle(self.indices)
        self.current_idx = 0
        return self

    def __next__(self):
        if self.current_idx >= self.num_samples:
            raise StopIteration

        start_idx = self.current_idx
        end_idx = start_idx + self.batch_size
        self.current_idx = end_idx

        batch_indices = self.indices[start_idx:end_idx]
        return self.features[batch_indices], self.labels[batch_indices]

    def __len__(self):
        return math.ceil(self.num_samples / self.batch_size)

# features = game_vectors (앞에서 만든 벡터)
# labels = df["game_id"] (혹은 df["name"])
loader = SimpleDataLoader(game_vectors, df["game_id"].values, batch_size=8, shuffle=True)

for batch_features, batch_labels in loader:
    print("Features shape:", batch_features.shape)
    print("Labels:", batch_labels)
    break  # 첫 번째 배치만 확인

import numpy as np

class GameEmbeddingModel:
    name = "game_embedding_model"

    def __init__(self, input_dim, hidden_dim, output_dim):
        # input_dim: 입력 피처 차원 (playtime, rating, owned_ratio + genre embedding 등)
        # hidden_dim: 은닉층 뉴런 수
        # output_dim: 최종 임베딩 차원 (예: 16)
        self.weights1 = np.random.randn(input_dim, hidden_dim) * 0.01
        self.bias1 = np.zeros((1, hidden_dim))
        self.weights2 = np.random.randn(hidden_dim, output_dim) * 0.01
        self.bias2 = np.zeros((1, output_dim))

    def relu(self, x):
        return np.maximum(0, x)

    def forward(self, x):
        # 은닉층
        self.z1 = np.dot(x, self.weights1) + self.bias1
        self.a1 = self.relu(self.z1)
        # 출력층 (Softmax 제거 → 점수/임베딩 벡터 그대로 반환)
        self.z2 = np.dot(self.a1, self.weights2) + self.bias2
        self.output = self.z2
        return self.output

    def backward(self, x, grad_output, lr=0.001):
        # grad_output: 외부에서 계산된 손실의 gradient
        m = len(x)

        dz2 = grad_output / m
        dw2 = np.dot(self.a1.T, dz2)
        db2 = np.sum(dz2, axis=0, keepdims=True)

        da1 = np.dot(dz2, self.weights2.T)
        dz1 = da1 * (self.z1 > 0)
        dw1 = np.dot(x.T, dz1)
        db1 = np.sum(dz1, axis=0, keepdims=True)

        # 가중치 업데이트
        self.weights2 -= lr * dw2
        self.bias2 -= lr * db2
        self.weights1 -= lr * dw1
        self.bias1 -= lr * db1


def train_embedding(model, train_loader, lr=0.001):
    total_loss = 0
    for features, labels in train_loader:
        embeddings = model.forward(features)  # [batch, emb_dim]

        # 코사인 유사도 행렬
        sims = cosine_similarity(embeddings)

        # 목표: 같은 배치 안에서 "자기 자신"과는 1, 다른 게임과는 0
        target = np.eye(len(labels))

        # MSE 손실 (유사도 vs 타겟)
        loss = np.mean((sims - target) ** 2)

        # gradient는 따로 계산해야 함 → grad_output 근사치로 (embeddings - target)
        grad_output = (embeddings - target.mean(axis=1, keepdims=True))

        model.backward(features, grad_output, lr)

        total_loss += loss

    return total_loss / len(train_loader)


def evaluate_embedding(model, val_loader):
    total_loss = 0.0
    all_similarities = []

    for features, labels in val_loader:
        # 1) 임베딩 추출
        embeddings = model.forward(features)   # [B, D]

        # 2) 코사인 유사도 행렬 계산
        sims = cosine_similarity(embeddings)  # [B, B]

        # 3) 타깃 행렬 (같은 장르는 1, 다르면 0)
        target = (labels[:, None] == labels[None, :]).astype(float)

        # 4) 손실 (MSE)
        diff = sims - target
        loss = np.mean(diff ** 2)

        total_loss += loss * len(features)

        # 5) 추천 후보 예시 (자기 자신 제외 Top-3)
        for i in range(len(labels)):
            sim_scores = sims[i]
            ranked_idx = sim_scores.argsort()[::-1][1:4]  # 자기 자신 제외
            all_similarities.append((labels[i], labels[ranked_idx]))

    avg_loss = total_loss / len(val_loader)
    return avg_loss, all_similarities

np.random.seed(42)

# features/labels 준비 (앞에서 만든 game_vectors, genre_encoded)

# CSV 불러오기
df = pd.read_csv("Top-40 Video Games.csv")

# genre → genre_encoded (숫자 인코딩)
le = LabelEncoder()
df["genre_encoded"] = le.fit_transform(df["genre"])

# features / labels 준비
features = game_vectors
labels = df["genre_encoded"].values   # 학습은 장르 기준

def train_embedding_similarity(model, train_loader, lr=1e-3):
    """
    모델 출력(임베딩) E로 유사도 행렬 S=E E^T를 만든 뒤
    장르 동일 여부로 만든 타깃 행렬 T와 MSE로 정합시키는 학습.
    """
    total_loss = 0.0
    for features, labels in train_loader:
        # 1) 순전파: 임베딩 추출
        E = model.forward(features)                # [B, D]

        # 2) 유사도 행렬 (dot product)
        S = E @ E.T

        # 3) 타깃 행렬 (같은 장르는 1, 다르면 0)
        T = (labels[:, None] == labels[None, :]).astype(float)

        # 4) 손실 (MSE)
        diff = S - T
        loss = np.mean(diff ** 2)

        # 5) 출력 임베딩에 대한 그래디언트: dL/dE = 4 * (S - T) * E / B
        B = len(labels)
        grad_E = (4.0 / B) * (diff @ E)

        # 6) 역전파 (가중치 업데이트)
        model.backward(features, grad_E, lr=lr)

        total_loss += loss

    return total_loss / len(train_loader)


# DataLoader 생성
train_loader = SimpleDataLoader(features, labels, batch_size=8, shuffle=True)
val_loader   = SimpleDataLoader(features, labels, batch_size=8, shuffle=False)

# 모델 초기화
model_params = {
    "input_dim": features.shape[1],   # 벡터 차원
    "hidden_dim": 64,
    "output_dim": 16                  # 최종 게임 임베딩 차원
}
model = GameEmbeddingModel(**model_params)

# 학습 루프
num_epochs = 10
for epoch in range(num_epochs):
    train_loss = train_embedding_similarity(model, train_loader, lr=1e-3)
    val_loss, examples = evaluate_embedding(model, val_loader)
    print(f"Epoch {epoch + 1}/{num_epochs}, "
          f"Train Loss: {train_loss:.4f}, "
          f"Val Loss: {val_loss:.4f}")

# 테스트 단계: 임베딩 뽑아서 추천 예시
embeddings = model.forward(features)
sims = cosine_similarity(embeddings)

# Devil May Cry 기준 Top-5 추천
target_game = "Devil May Cry 3: Dante's Awakening Special Edition"
idx = df[df["name"] == target_game].index[0]
sim_scores = sims[idx]
top_idx = sim_scores.argsort()[::-1][1:6]
print("추천 결과:")
print(df.iloc[top_idx][["name", "genre", "rating", "owned_ratio"]])


def model_save(model, model_params, epoch, loss, scaler=None, label_encoder=None, save_root="models"):
    """
    모델 가중치와 학습 관련 정보를 .pkl 파일로 저장하는 함수
    """
    # 저장 폴더 생성
    save_dir = os.path.join(save_root, model.name)
    os.makedirs(save_dir, exist_ok=True)

    # 파일명: epoch + 현재시간
    current_time = datetime.datetime.now().strftime("%y%m%d%H%M%S")
    dst = os.path.join(save_dir, f"E{epoch}_T{current_time}.pkl")

    # 저장 데이터
    save_data = {
        "epoch": epoch,
        "model_params": model_params,
        "model_state_dict": {
            "weights1": model.weights1,
            "bias1": model.bias1,
            "weights2": model.weights2,
            "bias2": model.bias2,
        },
        "loss": loss,
        "scaler": scaler,
        "label_encoder": label_encoder,
    }

    # 저장 실행
    with open(dst, "wb") as f:
        pickle.dump(save_data, f)

    print(f" Model saved to {dst}")


# ... 학습 수행 코드 (for epoch in range(num_epochs): ...)

# 모델 저장
model_save(
    model=model,
    model_params=model_params,
    epoch=num_epochs,
    loss=train_loss,         # 마지막 학습 손실
    scaler=scaler,           # 앞에서 StandardScaler()로 만든 객체
    label_encoder=le         # 앞에서 LabelEncoder()로 만든 객체
)


class CustomEnum(Enum):
    @classmethod
    def names(cls):
        """Enum 멤버 이름 리스트 반환"""
        return [member.name for member in list(cls)]

    @classmethod
    def validation(cls, name: str):
        """입력값이 Enum 멤버에 속하는지 검증"""
        names = [n.lower() for n in cls.names()]
        if name.lower() in names:
            return True
        else:
            raise ValueError(f" Invalid argument. Must be one of {cls.names()}")


class ModelTypes(CustomEnum):
    GAME_EMBEDDING = GameEmbeddingModel   # 게임 추천 모델


def run_train(model_name="game_embedding", num_epochs=10):
    # 1. 모델 이름 검증
    ModelTypes.validation(model_name)

    # 2. 데이터 준비
    df = pd.read_csv("Top-40 Video Games.csv")

    # genre 컬럼을 숫자로 변환 
    le = LabelEncoder()
    df["genre_encoded"] = le.fit_transform(df["genre"])

    # 수치형 피처 스케일링
    from sklearn.preprocessing import StandardScaler
    scaler = StandardScaler()
    scaled_features = scaler.fit_transform(df[["playtime", "rating", "owned_ratio"]])

    # 장르 임베딩 (이미 앞에서 짜둔 로직)
    embedding_dim = 4
    np.random.seed(42)
    genre_embeddings = np.random.randn(len(le.classes_), embedding_dim)
    genre_vectors = np.array([genre_embeddings[idx] for idx in df["genre_encoded"]])

    # 최종 feature 벡터
    game_vectors = np.hstack([genre_vectors, scaled_features])

    features = game_vectors
    labels = df["genre_encoded"].values

    # 3. DataLoader 생성
    train_loader = SimpleDataLoader(features, labels, batch_size=8, shuffle=True)
    val_loader   = SimpleDataLoader(features, labels, batch_size=8, shuffle=False)

    # 4. 모델 초기화
    model_class = ModelTypes[model_name.upper()].value
    model_params = {
        "input_dim": features.shape[1],
        "hidden_dim": 64,
        "output_dim": 16
    }
    model = model_class(**model_params)

    # 5. 학습 루프
    for epoch in range(num_epochs):
        train_loss = train_embedding_similarity(model, train_loader, lr=1e-3)
        val_loss, _ = evaluate_embedding(model, val_loader)
        print(f"Epoch {epoch+1}/{num_epochs} | Train Loss {train_loss:.4f} | Val Loss {val_loss:.4f}")

    # 6. 모델 저장
    model_save(model, model_params, epoch=num_epochs, loss=train_loss, scaler=scaler, label_encoder=le)

    print("Training finished! Model saved.")


#  최종 게임 벡터 (X)
game_vectors = np.hstack([genre_vectors, scaled_features])
X_train = game_vectors   # ✅ 학습 입력

# 라벨 (y)
y_train = df["genre_encoded"].values   # ✅ 장르 인덱스 (혹은 다른 목적 변수)

# 모델 초기화
model = GameEmbeddingModel(input_dim=X_train.shape[1], hidden_dim=32, output_dim=16)

# 8. 학습 루프
for epoch in range(20):
    outputs = model.forward(X_train)   # ✅ 이제 X_train 있음
    loss = np.mean((outputs - y_train.reshape(-1, 1))**2)  
    print(f"Epoch {epoch+1}, Loss: {loss:.4f}")

